In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from scipy.spatial.distance import cosine

## Load given data

In [2]:
M_sim = pd.DataFrame.from_csv('data/examples/example-mutation-counts.tsv', sep='\t')
M_sim.info()
M_sim.head()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, Sample-1 to Sample-100
Data columns (total 20 columns):
C1     100 non-null int64
C2     100 non-null int64
C3     100 non-null int64
C4     100 non-null int64
C5     100 non-null int64
C6     100 non-null int64
C7     100 non-null int64
C8     100 non-null int64
C9     100 non-null int64
C10    100 non-null int64
C11    100 non-null int64
C12    100 non-null int64
C13    100 non-null int64
C14    100 non-null int64
C15    100 non-null int64
C16    100 non-null int64
C17    100 non-null int64
C18    100 non-null int64
C19    100 non-null int64
C20    100 non-null int64
dtypes: int64(20)
memory usage: 16.4+ KB


,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20
Sample-1,5,10,2,0,9,9,18,0,0,0,0,9,12,22,1,10,13,6,6,16
Sample-2,3,6,4,1,13,27,15,0,0,0,0,4,12,19,1,4,26,3,2,8
Sample-3,0,3,2,0,3,3,34,0,0,0,0,15,2,6,0,9,3,1,15,8
Sample-4,3,6,11,0,7,22,35,0,0,0,0,9,7,18,2,9,10,16,4,91
Sample-5,6,13,9,1,28,13,6,0,0,0,1,2,10,25,1,2,13,11,0,21


In [3]:
signatures_sim = np.load('data/examples/example-signatures.npy')
print(signatures_sim.shape)
signatures_sim

(5, 20)


array([[  5.38459936e-05,   4.97804468e-06,   1.10683029e-01,
          3.95227089e-05,   1.88580413e-03,   1.07407637e-05,
          1.61530727e-01,   3.80624466e-04,   1.39435612e-02,
          1.62568978e-06,   1.65725644e-03,   2.12925065e-04,
          7.56245822e-11,   1.94127286e-04,   1.37872994e-02,
          1.21286224e-04,   2.82828314e-05,   1.36266770e-01,
          2.07835616e-05,   5.59176809e-01],
       [  7.37372068e-02,   1.63485357e-01,   2.05694148e-02,
          2.08423760e-13,   1.31158291e-01,   1.12889687e-08,
          2.50985574e-02,   1.64325173e-26,   3.38577083e-03,
          5.03798773e-07,   2.31541674e-11,   5.19539817e-15,
          1.11073641e-01,   2.79931424e-01,   1.04654938e-03,
          1.28629423e-02,   9.57385145e-02,   3.97023631e-02,
          6.17895305e-06,   4.22032740e-02],
       [  1.16776465e-03,   7.03014360e-19,   1.42158643e-04,
          3.45813689e-04,   1.02732058e-02,   5.37433438e-01,
          2.70858204e-03,   6.35366877e-03

In [4]:
##The dimensions of M_sim (G x K) are a transpose of the catalog dimensions (K x G) given in the paper
##We have number of signatures (N) = 5

## NMF and Iterate

In [5]:
##Each NMF run has a max iterations of 10000, as in the paper, using a convergence tolerance of 10^-6,
## and random intialization
##Each NMF is iterated for 100 times

##Input:
##M_sim.T ==> mutation catalog
##N ==> no. of signatures (hyper-parameter)

##Output:
##P_list_simulated: list of P matrices generated (matrix of mutation signature probabilities- dimensions: K x N)
##E_list_simulated: list of E matrices generated (matrix of mutation exposures- dimensions: N x G)

P_list_simulated = []
E_list_simulated = []
for i in range(100):
    print(i)
    model = NMF(n_components=5, init='random', max_iter=10000, tol=1e-6)
    P = model.fit_transform(M_sim.T)
    E = model.components_
    P_list_simulated.append(P)
    E_list_simulated.append(E)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


## Cluster

In [6]:
P_list_sim_concatenated = np.concatenate(P_list_simulated, axis=1)
print(P_list_sim_concatenated.shape)
P_list_sim_concatenated

(20, 500)


array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.83448412e-01,   0.00000000e+00,   0.00000000e+00],
       [  1.69702780e-01,   0.00000000e+00,   0.00000000e+00, ...,
          4.77595155e-01,   0.00000000e+00,   1.41339366e-01],
       [  2.42734350e+00,   2.78112335e-01,   0.00000000e+00, ...,
          4.35038866e+00,   1.60767887e-01,   2.02306898e+00],
       ..., 
       [  3.05110321e+00,   3.38959990e-01,   3.86287670e-02, ...,
          1.92380367e-02,   1.95571617e-01,   2.54282624e+00],
       [  7.70006533e-03,   4.04340001e+00,   1.66589861e-01, ...,
          0.00000000e+00,   2.33311161e+00,   6.41231689e-03],
       [  1.38924472e+01,   0.00000000e+00,   1.34930788e+00, ...,
          1.26626025e-01,   0.00000000e+00,   1.15782369e+01]])

In [7]:
clusterer = KMeans(n_clusters=5)
clusterer_labels = clusterer.fit_predict(P_list_sim_concatenated.T)
len(clusterer_labels)

500

In [8]:
Counter(clusterer_labels)
#Partitioning the signatures from previous iterations so that each signature from every P is assigned
# to exactly 1 cluster 

Counter({0: 100, 1: 100, 2: 100, 3: 100, 4: 100})

In [9]:
##Compute cosine similarity for signatures within each cluster
cos_sim_list = [cosine_similarity(P_list_sim_concatenated.T[clusterer_labels==label])
                for label in np.unique(clusterer_labels)]
print(len(cos_sim_list))
[(np.mean(cslist, axis=0), np.mean(cslist, axis=1), np.median(cslist, axis=0), np.median(cslist, axis=1))
 for cslist in cos_sim_list]
##The outputs for clusters associated with each of the 5 signatures show that both, columwise and rowwise, means and
## and medians have value ~1, i.e. the signatures are quite similar

5


[(array([ 0.99999999,  0.99999999,  0.99999999,  0.99999997,  0.99999998,
          0.99999997,  0.99999998,  0.99999999,  0.99999999,  0.99999999,
          0.99999998,  0.99999999,  0.99999998,  0.99999997,  0.99999999,
          0.99999999,  0.99999997,  0.99999999,  0.99999999,  0.99999999,
          0.99999999,  0.99999997,  0.99999999,  0.99999998,  0.99999999,
          0.99999999,  0.99999999,  0.99999998,  0.99999998,  0.99999998,
          0.99999999,  0.99999999,  0.99999997,  0.99999999,  0.99999998,
          0.99999999,  0.99999999,  0.99999999,  0.99999999,  0.99999998,
          0.99999998,  0.99999998,  0.99999998,  0.99999999,  0.99999999,
          0.99999999,  0.99999999,  0.99999998,  0.99999999,  0.99999999,
          0.99999999,  0.99999999,  0.99999999,  0.99999999,  0.99999999,
          0.99999999,  0.99999999,  0.99999999,  0.99999999,  0.99999999,
          0.99999999,  0.99999999,  0.99999999,  0.99999999,  0.99999999,
          0.99999999,  0.99999999,  0.

In [10]:
sig_mean = [np.mean(P_list_sim_concatenated.T[clusterer_labels==label], axis=0)
            for label in np.unique(clusterer_labels)]

In [11]:
sig_mean = np.array(sig_mean)
print(sig_mean.shape)
sig_mean

(5, 20)


array([[  1.94040004e-01,   5.05202091e-01,   4.60348655e+00,
          2.08263438e-01,   1.15642274e+01,   1.44130662e+00,
          0.00000000e+00,   0.00000000e+00,   9.35099187e-02,
          1.16136771e+00,   4.29211257e-02,   2.63189111e-01,
          0.00000000e+00,   4.46251157e-02,   2.57504279e-01,
          2.84691962e-01,   4.69374102e-02,   2.03143438e-02,
          0.00000000e+00,   1.34233352e-01],
       [  0.00000000e+00,   1.62693522e-01,   2.32758459e+00,
          0.00000000e+00,   0.00000000e+00,   1.30799380e+00,
          4.44385248e+00,   0.00000000e+00,   1.93955364e-01,
          0.00000000e+00,   9.96699005e-02,   0.00000000e+00,
          1.61743178e-01,   7.23323866e-02,   4.36462338e-01,
          2.95027862e-01,   0.00000000e+00,   2.92566733e+00,
          7.38158251e-03,   1.33213437e+01],
       [  0.00000000e+00,   0.00000000e+00,   2.13585159e-01,
          0.00000000e+00,   1.97380346e-01,   0.00000000e+00,
          9.27454668e+00,   1.71282004e-02

## Evaluate

In [12]:
cosine(signatures_sim.reshape(-1,1), np.array(sig_mean).reshape(-1,1))
##Cosine similarity value close to 1 resembles high similarity between 'true' and observed signatures

0.91337758866131724

In [13]:
##visualizing the cluster labels mapping for the signatures generated through the 100 iterations:
[np.where(clusterer_labels==label) for label in np.unique(clusterer_labels)]
##It can be seen that each value in blocks of 5 numbers (i.e. 0-4, 5-9,...) are mapped to a distinct cluster

[(array([  4,   8,  10,  17,  21,  29,  32,  35,  40,  46,  53,  59,  61,
          68,  74,  76,  84,  87,  91,  95, 100, 106, 112, 117, 120, 128,
         131, 136, 144, 147, 150, 156, 164, 168, 172, 177, 182, 187, 190,
         197, 201, 206, 211, 216, 224, 228, 233, 237, 241, 245, 250, 257,
         262, 266, 271, 278, 280, 287, 293, 299, 304, 308, 311, 316, 323,
         329, 331, 337, 344, 346, 352, 358, 362, 365, 371, 376, 382, 389,
         390, 399, 402, 406, 411, 418, 422, 426, 433, 436, 441, 448, 452,
         459, 460, 468, 472, 477, 480, 486, 490, 497]),),
 (array([  0,   6,  12,  18,  24,  25,  33,  39,  41,  45,  52,  56,  60,
          66,  73,  75,  80,  85,  92,  97, 102, 109, 113, 119, 122, 127,
         130, 139, 140, 146, 151, 157, 160, 165, 170, 178, 180, 188, 194,
         199, 200, 205, 210, 219, 220, 225, 230, 235, 240, 249, 254, 255,
         263, 265, 273, 277, 282, 288, 291, 295, 303, 305, 312, 315, 324,
         327, 330, 335, 342, 349, 350, 359, 363, 367, 

In [14]:
cosine_similarity(signatures_sim, sig_mean)
##Following output shows that the signature identified by cluster with label 0 is similar to the 2nd 'true' signature
##and so on..
##True_signature_0 <==> predicted_signature_cluster_1
##True_signature_1 <==> predicted_signature_cluster_3
##True_signature_2 <==> predicted_signature_cluster_4
##True_signature_3 <==> predicted_signature_cluster_0
##True_signature_4 <==> predicted_signature_cluster_2

array([[ 0.08009941,  0.99425935,  0.22546124,  0.13968838,  0.10317496],
       [ 0.35110957,  0.15843953,  0.16461295,  0.99038781,  0.15638886],
       [ 0.12204176,  0.26081711,  0.00976969,  0.16608456,  0.99639266],
       [ 0.99358863,  0.07055942,  0.03270132,  0.23582456,  0.20476886],
       [ 0.01761582,  0.25106781,  0.99880907,  0.11915806,  0.00851738]])